<a href="https://colab.research.google.com/github/ameerfayiz/ML-notebooks/blob/main/Zero_shot_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.5 MB/s eta 0:00:00


In [ ]:
# set labels

better_day = ["exercise", "breakfast", "lunch","dinner", "music", "water", "sleep", "relaxation", "nature", "happiness", "creativity", "kindness", "gratitude", "laughter", "adventure", "learning", "compassion", "love", "inspiration", "mindfulness"]
worse_day = ["stress", "hunger", "pain", "illness", "conflict", "traffic", "workload", "loneliness", "boredom", "tiredness", "anxiety", "depression", "weather", "noise", "crowds", "pollution", "worries", "deadlines", "disrespect", "negativity"]

#labels = ['travel', 'friends','series','movies','entertainment','technology','family' ,'algorithms','cooking', 'job' , 'dancing','health','happiness','activity','social']
labels = ['algorithms', 'job', 'meeting','phone call','video call','social interaction','outdoors' ,'coding', 'video editing', 'hospital', 'documentary', 'movies', 'hiking', 'youtube', 'cooking', 'reddit', 'friends', 'audio books', 'research', 'video posting', 'email', 'love', 'songs', 'reading', 'cleaning', 'health', 'gaming', 'fasting', 'dancing', 'shopping', 'art', 'series', 'streaming', 'quran', 'designing', 'dairy', 'meditation', 'entertainment', 'exercise', 'travel', 'power nap', 'writing', 'yoga', 'walk', 'learning', 'tutorial', 'news update', 'technology', 'happiness', 'party', 'shower', 'family', 'prayer', 'podcast', 'repair', 'food']

labels+=worse_day
labels+=better_day
labels= list(set(labels))

print(labels)

['worries', 'illness', 'stress', 'podcast', 'creativity', 'tiredness', 'job', 'cooking', 'dairy', 'family', 'pain', 'disrespect', 'prayer', 'tutorial', 'adventure', 'dinner', 'inspiration', 'writing', 'video call', 'algorithms', 'shower', 'anxiety', 'dancing', 'boredom', 'deadlines', 'conflict', 'fasting', 'friends', 'compassion', 'loneliness', 'traffic', 'youtube', 'water', 'mindfulness', 'party', 'streaming', 'video editing', 'exercise', 'weather', 'designing', 'email', 'power nap', 'technology', 'kindness', 'sleep', 'nature', 'pollution', 'workload', 'travel', 'gaming', 'negativity', 'laughter', 'quran', 'yoga', 'food', 'love', 'hiking', 'news update', 'entertainment', 'relaxation', 'repair', 'movies', 'learning', 'documentary', 'shopping', 'crowds', 'phone call', 'breakfast', 'art', 'gratitude', 'research', 'depression', 'music', 'meeting', 'video posting', 'hospital', 'reading', 'meditation', 'hunger', 'health', 'series', 'reddit', 'social interaction', 'audio books', 'songs', 'ou

In [ ]:
import json
from transformers import pipeline
import time
import torch

torch.cuda.is_available()


# classifier = pipeline ("zero-shot-classification", model = "facebook/bart-large-mnli")                  # 1.63 Gb 0.4384075531592736 per label
# classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")      # 558  Mb 0.313440519112807 per label
# classifier = pipeline("zero-shot-classification", model="BaptisteDoyen/camembert-base-xnli")            # 443 Mb 0.12476613338177021 per label
# classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-1")                # 890 Mb 0.20728054413428673 per label 
# classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli") # 369 Mb 0.3029702296623817 per label 
# classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-v3-base")            # 738 Mb 0.30590915496532733 per label
# classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-v3-small")           # 568 Mb 0.15964309252225434 per label
classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-MiniLM2-L6-H768")            # 329 Mb 0.055275693306556115 per label
# classifier = pipeline("zero-shot-classification", model="typeform/distilbert-base-uncased-mnli")        # 268 Mb 0.05824783215155968 per label
# classifier = pipeline("zero-shot-classification", model="typeform/mobilebert-uncased-mnli")             # 99 Mb 0.03554045275637978 per label
# classifier = pipeline("zero-shot-classification", model="Recognai/zeroshot_selectra_medium")            # 163 Mb 0.03785876310788668 per label
# classifier = pipeline("zero-shot-classification", model="Recognai/zeroshot_selectra_small")             # 90 Mb 0.019344867978777207 per label



print("model loaded!")


def classify(prompt,multi_label=True,loop=10,_labels=labels):
    begin = time.time()
    for i in range(loop):
      result = classifier(prompt, _labels,multi_label=multi_label)
    end = time.time()
    # print(json.dumps(result,indent="  "))
    # print(result['labels'])
    # print([round(x,2) for x in result['scores']])
    print({k:round(v,2) for k,v in zip(result['labels'], result['scores'])})
    # total time taken
    gap = (end - begin)/10
    print(f"Total runtime of the program is {gap} is, {gap/len(_labels)} per label")




model loaded!


In [ ]:
classify("To avoid plagiarism issues, please refer to the code on the Google Colab",loop=1)


['coding', 'technology', 'activity', 'exercise', 'worries', 'conflict', 'reading', 'shopping', 'writing', 'series', 'tutorial', 'algorithms', 'stress', 'job', 'recording', 'outdoors', 'research', 'deadlines', 'learning', 'music', 'workload', 'songs', 'pain', 'negativity', 'disrespect', 'streaming', 'designing', 'anxiety', 'noise', 'audio books', 'traffic', 'video editing', 'crowds', 'inspiration', 'travel', 'gaming', 'movies', 'walk', 'family', 'video call', 'email', 'shower', 'tiredness', 'quran', 'news update', 'jobs', 'nature', 'weather', 'social interaction', 'hospital', 'party', 'podcast', 'food', 'dairy', 'cleaning', 'creativity', 'repair', 'art', 'illness', 'adventure', 'water', 'youtube', 'loneliness', 'phone call', 'health', 'hiking', 'relaxation', 'documentary', 'friends', 'hunger', 'depression', 'pollution', 'boredom', 'breakfast', 'dancing', 'entertainment', 'reddit', 'cooking', 'prayer', 'compassion', 'gratitude', 'love', 'power nap', 'yoga', 'kindness', 'mindfulness', 'fa

In [ ]:
classify("Got up early at 04:40 to drive my families to the airport Went back to sleep til 09:00",loop=1)

['activity', 'sleep', 'social interaction', 'family', 'weather', 'stress', 'conflict', 'pain', 'learning', 'tiredness', 'loneliness', 'depression', 'noise', 'boredom', 'documentary', 'entertainment', 'disrespect', 'negativity', 'workload', 'inspiration', 'traffic', 'anxiety', 'travel', 'relaxation', 'outdoors', 'worries', 'exercise', 'mindfulness', 'illness', 'health', 'gratitude', 'meditation', 'phone call', 'adventure', 'recording', 'reading', 'kindness', 'hiking', 'algorithms', 'jobs', 'compassion', 'music', 'tutorial', 'writing', 'repair', 'fasting', 'deadlines', 'crowds', 'prayer', 'job', 'happiness', 'hunger', 'series', 'shower', 'cleaning', 'coding', 'sunshine', 'designing', 'party', 'cooking', 'pollution', 'songs', 'video call', 'video editing', 'news update', 'hospital', 'power nap', 'water', 'movies', 'shopping', 'research', 'creativity', 'love', 'walk', 'laughter', 'quran', 'art', 'breakfast', 'dairy', 'streaming', 'technology', 'audio books', 'nature', 'friends', 'yoga', 'p

In [ ]:
classify("Watched Netflix series Kaleidiscope with family on beach",loop=1)

['outdoors', 'entertainment', 'streaming', 'social interaction', 'series', 'family', 'activity', 'relaxation', 'technology', 'travel', 'adventure', 'happiness', 'kindness', 'inspiration', 'video call', 'water', 'sunshine', 'learning', 'compassion', 'gratitude', 'love', 'weather', 'nature', 'noise', 'friends', 'tutorial', 'workload', 'video editing', 'laughter', 'songs', 'creativity', 'party', 'tiredness', 'news update', 'worries', 'movies', 'documentary', 'repair', 'food', 'exercise', 'deadlines', 'youtube', 'recording', 'stress', 'music', 'health', 'conflict', 'hunger', 'writing', 'prayer', 'crowds', 'walk', 'algorithms', 'anxiety', 'dairy', 'mindfulness', 'podcast', 'negativity', 'dancing', 'gaming', 'job', 'quran', 'phone call', 'pain', 'boredom', 'illness', 'reading', 'disrespect', 'shower', 'art', 'cooking', 'cleaning', 'depression', 'breakfast', 'designing', 'loneliness', 'jobs', 'power nap', 'coding', 'research', 'fasting', 'audio books', 'yoga', 'meditation', 'email', 'sleep', 

In [ ]:
# labels = ['algorithms', 'job', 'phone call','video call','social interaction','outdoors' ,'coding', 'video editing', 'hospital', 'documentary', 'movies', 'hiking', 'youtube', 'cooking', 'reddit', 'friends', 'audio books', 'research', 'recording', 'email', 'love', 'songs', 'reading', 'social', 'cleaning', 'health', 'gaming', 'fasting', 'dancing', 'shopping', 'jobs', 'art', 'series', 'streaming', 'activity', 'quran', 'designing', 'dairy', 'meditation', 'entertainment', 'exercise', 'travel', 'power nap', 'writing', 'yoga', 'walk', 'learning', 'tutorial', 'news update', 'technology', 'happiness', 'party', 'shower', 'family', 'prayer', 'podcast', 'repair', 'food']

In [ ]:
classify("got little sick, but my lover was with me who made my day awesome",loop=1)

['social interaction', 'love', 'activity', 'happiness', 'kindness', 'gratitude', 'illness', 'inspiration', 'friends', 'entertainment', 'outdoors', 'learning', 'recording', 'series', 'tiredness', 'compassion', 'family', 'tutorial', 'weather', 'streaming', 'writing', 'adventure', 'workload', 'noise', 'job', 'party', 'songs', 'repair', 'music', 'health', 'news update', 'deadlines', 'phone call', 'hospital', 'documentary', 'creativity', 'dairy', 'email', 'jobs', 'travel', 'video call', 'conflict', 'prayer', 'reading', 'anxiety', 'exercise', 'breakfast', 'worries', 'sunshine', 'youtube', 'walk', 'art', 'mindfulness', 'nature', 'laughter', 'reddit', 'crowds', 'podcast', 'hiking', 'pain', 'algorithms', 'quran', 'movies', 'video editing', 'audio books', 'relaxation', 'gaming', 'hunger', 'food', 'yoga', 'research', 'shower', 'stress', 'water', 'designing', 'technology', 'shopping', 'coding', 'depression', 'cooking', 'meditation', 'traffic', 'dancing', 'disrespect', 'fasting', 'pollution', 'clea

In [ ]:
classify("fasted today, after prayer , i recited some quran",loop=1)

['quran', 'fasting', 'breakfast', 'prayer', 'meditation', 'inspiration', 'learning', 'activity', 'reading', 'series', 'outdoors', 'recording', 'tutorial', 'gratitude', 'mindfulness', 'tiredness', 'stress', 'workload', 'family', 'compassion', 'happiness', 'friends', 'anxiety', 'hunger', 'health', 'worries', 'writing', 'loneliness', 'kindness', 'conflict', 'illness', 'repair', 'weather', 'pain', 'disrespect', 'negativity', 'depression', 'nature', 'deadlines', 'crowds', 'love', 'hospital', 'walk', 'water', 'travel', 'social interaction', 'boredom', 'pollution', 'job', 'algorithms', 'exercise', 'party', 'cleaning', 'dairy', 'noise', 'adventure', 'food', 'songs', 'shower', 'designing', 'streaming', 'traffic', 'creativity', 'relaxation', 'jobs', 'hiking', 'technology', 'music', 'cooking', 'audio books', 'research', 'coding', 'shopping', 'art', 'youtube', 'video call', 'news update', 'sunshine', 'email', 'documentary', 'reddit', 'video editing', 'dancing', 'movies', 'power nap', 'yoga', 'ente

In [ ]:
classify("did some shopping for dinner",loop=1)

['activity', 'shopping', 'food', 'social interaction', 'workload', 'outdoors', 'recording', 'inspiration', 'tutorial', 'series', 'crowds', 'kindness', 'family', 'learning', 'stress', 'gratitude', 'deadlines', 'compassion', 'job', 'worries', 'happiness', 'jobs', 'creativity', 'friends', 'weather', 'tiredness', 'travel', 'dairy', 'entertainment', 'noise', 'hunger', 'quran', 'anxiety', 'adventure', 'exercise', 'sunshine', 'walk', 'cooking', 'nature', 'mindfulness', 'songs', 'health', 'documentary', 'disrespect', 'negativity', 'conflict', 'writing', 'boredom', 'pain', 'algorithms', 'loneliness', 'repair', 'reddit', 'technology', 'love', 'prayer', 'depression', 'news update', 'youtube', 'video call', 'streaming', 'party', 'relaxation', 'water', 'email', 'research', 'designing', 'podcast', 'illness', 'art', 'traffic', 'audio books', 'music', 'reading', 'laughter', 'movies', 'coding', 'video editing', 'pollution', 'phone call', 'cleaning', 'meditation', 'fasting', 'hospital', 'yoga', 'gaming'

In [ ]:
classify("Worked on a new project from 17:00 to 19:00",loop=1)

['activity', 'workload', 'job', 'deadlines', 'outdoors', 'designing', 'research', 'series', 'technology', 'repair', 'creativity', 'inspiration', 'learning', 'art', 'coding', 'algorithms', 'writing', 'video editing', 'cleaning', 'songs', 'stress', 'hospital', 'water', 'recording', 'worries', 'documentary', 'audio books', 'tutorial', 'tiredness', 'health', 'dairy', 'adventure', 'noise', 'music', 'exercise', 'gaming', 'hiking', 'travel', 'weather', 'reddit', 'yoga', 'movies', 'walk', 'social interaction', 'streaming', 'kindness', 'quran', 'gratitude', 'compassion', 'shower', 'crowds', 'traffic', 'meditation', 'youtube', 'food', 'anxiety', 'entertainment', 'family', 'pain', 'video call', 'conflict', 'pollution', 'happiness', 'nature', 'mindfulness', 'reading', 'illness', 'news update', 'hunger', 'sunshine', 'friends', 'email', 'podcast', 'love', 'prayer', 'cooking', 'shopping', 'negativity', 'disrespect', 'dancing', 'loneliness', 'phone call', 'boredom', 'fasting', 'depression', 'party', '

In [ ]:
classify("Had a meeting with a client at 11:00",loop=1)

['meeting', 'activity', 'social interaction', 'workload', 'learning', 'recording', 'inspiration', 'noise', 'outdoors', 'deadlines', 'stress', 'series', 'worries', 'job', 'kindness', 'writing', 'repair', 'compassion', 'anxiety', 'happiness', 'tiredness', 'gratitude', 'tutorial', 'news update', 'designing', 'conflict', 'relaxation', 'illness', 'exercise', 'health', 'audio books', 'negativity', 'video editing', 'reading', 'hospital', 'coding', 'walk', 'weather', 'pain', 'depression', 'email', 'mindfulness', 'family', 'technology', 'quran', 'entertainment', 'sunshine', 'disrespect', 'phone call', 'crowds', 'boredom', 'travel', 'hunger', 'reddit', 'algorithms', 'prayer', 'love', 'friends', 'songs', 'documentary', 'dairy', 'research', 'art', 'water', 'shower', 'music', 'video call', 'food', 'nature', 'creativity', 'meditation', 'loneliness', 'shopping', 'traffic', 'youtube', 'adventure', 'streaming', 'gaming', 'party', 'movies', 'breakfast', 'laughter', 'pollution', 'podcast', 'cleaning', 'y

In [ ]:
classify('went for a walk by the river',loop=1)

['outdoors', 'activity', 'walk', 'exercise', 'water', 'hiking', 'travel', 'nature', 'adventure', 'weather', 'social interaction', 'relaxation', 'kindness', 'entertainment', 'tutorial', 'noise', 'recording', 'inspiration', 'happiness', 'gratitude', 'learning', 'worries', 'tiredness', 'compassion', 'friends', 'family', 'sunshine', 'stress', 'series', 'workload', 'health', 'streaming', 'writing', 'job', 'repair', 'anxiety', 'loneliness', 'creativity', 'documentary', 'deadlines', 'meeting', 'love', 'songs', 'boredom', 'mindfulness', 'hunger', 'disrespect', 'designing', 'news update', 'prayer', 'research', 'art', 'video call', 'crowds', 'quran', 'negativity', 'conflict', 'music', 'party', 'pain', 'algorithms', 'depression', 'food', 'podcast', 'dairy', 'pollution', 'illness', 'reading', 'breakfast', 'cleaning', 'laughter', 'coding', 'youtube', 'fasting', 'shopping', 'video editing', 'shower', 'meditation', 'technology', 'email', 'audio books', 'reddit', 'yoga', 'traffic', 'phone call', 'hosp

In [ ]:
classify('Did some work on my website',loop=1)

{'activity': 1.0, 'workload': 0.99, 'technology': 0.98, 'creativity': 0.91, 'social interaction': 0.88, 'learning': 0.87, 'job': 0.72, 'reading': 0.71, 'inspiration': 0.71, 'writing': 0.69, 'repair': 0.68, 'designing': 0.65, 'coding': 0.54, 'news update': 0.48, 'deadlines': 0.43, 'exercise': 0.39, 'series': 0.38, 'stress': 0.34, 'worries': 0.33, 'tutorial': 0.3, 'tiredness': 0.29, 'noise': 0.26, 'art': 0.21, 'outdoors': 0.2, 'algorithms': 0.19, 'reddit': 0.18, 'kindness': 0.14, 'gratitude': 0.13, 'research': 0.13, 'cleaning': 0.13, 'compassion': 0.12, 'video editing': 0.1, 'recording': 0.1, 'negativity': 0.1, 'conflict': 0.1, 'traffic': 0.09, 'anxiety': 0.08, 'friends': 0.08, 'entertainment': 0.08, 'shopping': 0.08, 'happiness': 0.08, 'mindfulness': 0.07, 'quran': 0.06, 'family': 0.06, 'adventure': 0.06, 'health': 0.06, 'weather': 0.05, 'songs': 0.04, 'disrespect': 0.04, 'love': 0.04, 'pain': 0.04, 'music': 0.04, 'walk': 0.03, 'prayer': 0.03, 'depression': 0.03, 'streaming': 0.03, 'sun

In [ ]:
classify('Went to Per Ivars 30th birthday party',loop=1)

{'activity': 1.0, 'party': 1.0, 'social interaction': 1.0, 'gratitude': 0.99, 'entertainment': 0.98, 'outdoors': 0.82, 'music': 0.82, 'happiness': 0.81, 'inspiration': 0.73, 'kindness': 0.7, 'songs': 0.59, 'friends': 0.58, 'noise': 0.52, 'recording': 0.46, 'family': 0.42, 'series': 0.28, 'learning': 0.27, 'crowds': 0.26, 'laughter': 0.24, 'love': 0.2, 'walk': 0.19, 'compassion': 0.19, 'tutorial': 0.14, 'travel': 0.14, 'dancing': 0.13, 'creativity': 0.12, 'food': 0.11, 'art': 0.1, 'meeting': 0.09, 'relaxation': 0.08, 'mindfulness': 0.08, 'adventure': 0.07, 'stress': 0.07, 'writing': 0.07, 'documentary': 0.06, 'worries': 0.06, 'dinner': 0.06, 'streaming': 0.05, 'water': 0.05, 'movies': 0.05, 'workload': 0.05, 'dairy': 0.05, 'weather': 0.05, 'deadlines': 0.05, 'job': 0.04, 'exercise': 0.04, 'disrespect': 0.04, 'repair': 0.04, 'health': 0.03, 'tiredness': 0.03, 'video call': 0.03, 'youtube': 0.03, 'algorithms': 0.02, 'reddit': 0.02, 'nature': 0.02, 'shower': 0.02, 'negativity': 0.02, 'anxi

In [ ]:
classify("Went for a hike in the mountains",loop=1)

{'outdoors': 1.0, 'activity': 1.0, 'exercise': 1.0, 'hiking': 1.0, 'gratitude': 0.99, 'walk': 0.99, 'nature': 0.99, 'travel': 0.98, 'weather': 0.98, 'adventure': 0.98, 'mindfulness': 0.82, 'inspiration': 0.81, 'recording': 0.81, 'tutorial': 0.75, 'kindness': 0.7, 'noise': 0.69, 'tiredness': 0.66, 'learning': 0.64, 'stress': 0.64, 'happiness': 0.57, 'entertainment': 0.55, 'worries': 0.54, 'workload': 0.44, 'series': 0.42, 'health': 0.38, 'friends': 0.31, 'social interaction': 0.3, 'anxiety': 0.3, 'meditation': 0.29, 'prayer': 0.29, 'writing': 0.29, 'family': 0.28, 'job': 0.25, 'compassion': 0.25, 'deadlines': 0.19, 'hunger': 0.16, 'streaming': 0.13, 'meeting': 0.09, 'documentary': 0.08, 'relaxation': 0.08, 'songs': 0.08, 'love': 0.07, 'crowds': 0.07, 'news update': 0.05, 'disrespect': 0.05, 'conflict': 0.05, 'creativity': 0.04, 'pain': 0.04, 'repair': 0.04, 'research': 0.04, 'music': 0.03, 'loneliness': 0.03, 'algorithms': 0.03, 'negativity': 0.03, 'art': 0.03, 'boredom': 0.03, 'video c

In [ ]:
classify("Posted a members video on youtube",loop=1)

{'youtube': 1.0, 'video posting': 0.97, 'activity': 0.93, 'technology': 0.9, 'streaming': 0.87, 'family': 0.83, 'social interaction': 0.81, 'friends': 0.81, 'entertainment': 0.79, 'inspiration': 0.69, 'creativity': 0.44, 'video editing': 0.43, 'tutorial': 0.39, 'series': 0.38, 'video call': 0.35, 'meeting': 0.35, 'learning': 0.31, 'kindness': 0.25, 'outdoors': 0.24, 'music': 0.23, 'songs': 0.2, 'gratitude': 0.19, 'worries': 0.18, 'compassion': 0.18, 'adventure': 0.17, 'happiness': 0.17, 'relaxation': 0.15, 'gaming': 0.14, 'workload': 0.14, 'deadlines': 0.13, 'documentary': 0.12, 'movies': 0.12, 'noise': 0.1, 'laughter': 0.08, 'prayer': 0.08, 'stress': 0.08, 'mindfulness': 0.08, 'tiredness': 0.08, 'exercise': 0.08, 'conflict': 0.08, 'love': 0.08, 'weather': 0.07, 'boredom': 0.07, 'news update': 0.07, 'party': 0.06, 'job': 0.06, 'travel': 0.06, 'walk': 0.05, 'art': 0.05, 'quran': 0.04, 'anxiety': 0.04, 'dancing': 0.04, 'disrespect': 0.04, 'hunger': 0.04, 'crowds': 0.04, 'health': 0.04, '

In [ ]:
classify("Visited my sister and brother in law at 11:00",loop=1)

{'social interaction': 0.99, 'family': 0.99, 'activity': 0.99, 'travel': 0.97, 'meeting': 0.97, 'friends': 0.84, 'outdoors': 0.73, 'writing': 0.72, 'inspiration': 0.71, 'series': 0.68, 'learning': 0.6, 'kindness': 0.51, 'tutorial': 0.49, 'relaxation': 0.41, 'noise': 0.41, 'workload': 0.4, 'entertainment': 0.39, 'party': 0.36, 'deadlines': 0.35, 'compassion': 0.35, 'news update': 0.34, 'happiness': 0.34, 'gratitude': 0.32, 'walk': 0.26, 'stress': 0.24, 'crowds': 0.24, 'adventure': 0.24, 'worries': 0.23, 'exercise': 0.21, 'reading': 0.2, 'love': 0.17, 'prayer': 0.17, 'tiredness': 0.16, 'songs': 0.13, 'video posting': 0.13, 'streaming': 0.13, 'anxiety': 0.12, 'job': 0.11, 'weather': 0.11, 'dinner': 0.09, 'phone call': 0.09, 'music': 0.09, 'health': 0.08, 'repair': 0.08, 'documentary': 0.07, 'video editing': 0.07, 'shopping': 0.07, 'email': 0.06, 'mindfulness': 0.06, 'traffic': 0.06, 'video call': 0.06, 'laughter': 0.06, 'conflict': 0.05, 'youtube': 0.05, 'creativity': 0.05, 'food': 0.05, 

In [ ]:
classify("Did some webdesign work",loop=1)

{'activity': 0.99, 'technology': 0.99, 'designing': 0.96, 'workload': 0.9, 'learning': 0.8, 'creativity': 0.76, 'coding': 0.71, 'inspiration': 0.4, 'tutorial': 0.28, 'series': 0.27, 'reddit': 0.23, 'worries': 0.21, 'outdoors': 0.18, 'research': 0.16, 'streaming': 0.15, 'job': 0.13, 'algorithms': 0.13, 'deadlines': 0.12, 'social interaction': 0.11, 'youtube': 0.09, 'stress': 0.07, 'quran': 0.06, 'exercise': 0.04, 'meeting': 0.04, 'family': 0.03, 'conflict': 0.03, 'tiredness': 0.03, 'friends': 0.03, 'noise': 0.03, 'reading': 0.02, 'repair': 0.02, 'anxiety': 0.02, 'kindness': 0.02, 'news update': 0.02, 'happiness': 0.02, 'crowds': 0.02, 'compassion': 0.02, 'art': 0.02, 'writing': 0.02, 'gratitude': 0.02, 'adventure': 0.01, 'walk': 0.01, 'pain': 0.01, 'relaxation': 0.01, 'love': 0.01, 'negativity': 0.01, 'entertainment': 0.01, 'disrespect': 0.01, 'party': 0.01, 'mindfulness': 0.01, 'loneliness': 0.01, 'traffic': 0.01, 'video posting': 0.01, 'boredom': 0.01, 'hunger': 0.0, 'video editing': 

In [ ]:
classify("I attended a traditional Indian wedding ceremony with my cousin in Jaipur",loop=1)

{'family': 0.99, 'social interaction': 0.97, 'outdoors': 0.94, 'inspiration': 0.85, 'tutorial': 0.81, 'friends': 0.69, 'entertainment': 0.67, 'series': 0.65, 'gratitude': 0.61, 'writing': 0.6, 'learning': 0.57, 'prayer': 0.5, 'love': 0.48, 'meeting': 0.48, 'kindness': 0.41, 'songs': 0.39, 'happiness': 0.38, 'music': 0.33, 'compassion': 0.31, 'worries': 0.29, 'workload': 0.28, 'party': 0.25, 'video posting': 0.24, 'quran': 0.23, 'noise': 0.22, 'dancing': 0.13, 'crowds': 0.1, 'weather': 0.1, 'stress': 0.1, 'streaming': 0.09, 'walk': 0.09, 'deadlines': 0.08, 'travel': 0.07, 'repair': 0.07, 'nature': 0.06, 'exercise': 0.05, 'relaxation': 0.05, 'documentary': 0.05, 'health': 0.05, 'algorithms': 0.04, 'news update': 0.04, 'shower': 0.03, 'water': 0.03, 'conflict': 0.03, 'art': 0.03, 'disrespect': 0.03, 'video call': 0.03, 'tiredness': 0.03, 'anxiety': 0.02, 'laughter': 0.02, 'podcast': 0.02, 'pain': 0.02, 'mindfulness': 0.01, 'dairy': 0.01, 'job': 0.01, 'food': 0.01, 'designing': 0.01, 'yout

In [ ]:
classify("I went to a jazz club with my friends",loop=1)

{'social interaction': 1.0, 'music': 1.0, 'entertainment': 1.0, 'friends': 1.0, 'songs': 0.99, 'noise': 0.94, 'inspiration': 0.9, 'outdoors': 0.84, 'tutorial': 0.81, 'learning': 0.79, 'party': 0.79, 'meeting': 0.78, 'gratitude': 0.71, 'happiness': 0.71, 'series': 0.71, 'kindness': 0.67, 'relaxation': 0.53, 'travel': 0.5, 'worries': 0.48, 'streaming': 0.41, 'adventure': 0.39, 'dancing': 0.38, 'creativity': 0.31, 'stress': 0.29, 'compassion': 0.27, 'workload': 0.22, 'tiredness': 0.2, 'crowds': 0.17, 'exercise': 0.16, 'writing': 0.13, 'video posting': 0.12, 'anxiety': 0.11, 'love': 0.08, 'deadlines': 0.08, 'repair': 0.07, 'conflict': 0.06, 'disrespect': 0.06, 'mindfulness': 0.05, 'walk': 0.05, 'negativity': 0.05, 'boredom': 0.04, 'news update': 0.03, 'laughter': 0.03, 'art': 0.03, 'weather': 0.03, 'job': 0.03, 'video call': 0.03, 'depression': 0.02, 'family': 0.02, 'prayer': 0.02, 'shower': 0.02, 'documentary': 0.02, 'pain': 0.02, 'quran': 0.02, 'lunch': 0.01, 'dinner': 0.01, 'hunger': 0.

In [ ]:
classify("got humiliated infront of public",loop=1)

{'disrespect': 0.99, 'social interaction': 0.97, 'negativity': 0.97, 'pain': 0.82, 'stress': 0.75, 'crowds': 0.75, 'tiredness': 0.69, 'conflict': 0.69, 'outdoors': 0.55, 'worries': 0.42, 'learning': 0.34, 'series': 0.33, 'noise': 0.29, 'depression': 0.25, 'anxiety': 0.18, 'streaming': 0.17, 'meeting': 0.17, 'power nap': 0.17, 'reading': 0.15, 'deadlines': 0.15, 'workload': 0.14, 'video posting': 0.12, 'tutorial': 0.12, 'writing': 0.12, 'news update': 0.11, 'video call': 0.09, 'pollution': 0.09, 'documentary': 0.08, 'shower': 0.08, 'job': 0.08, 'nature': 0.08, 'boredom': 0.07, 'walk': 0.07, 'reddit': 0.07, 'quran': 0.06, 'exercise': 0.05, 'illness': 0.05, 'travel': 0.04, 'breakfast': 0.04, 'email': 0.04, 'family': 0.03, 'hospital': 0.03, 'audio books': 0.03, 'songs': 0.03, 'movies': 0.03, 'technology': 0.03, 'phone call': 0.03, 'inspiration': 0.03, 'research': 0.02, 'party': 0.02, 'youtube': 0.02, 'dairy': 0.02, 'lunch': 0.02, 'algorithms': 0.02, 'coding': 0.02, 'video editing': 0.02, '

In [ ]:
classify("I took my car for a regular maintenance checkup",loop=1)

{'repair': 0.99, 'outdoors': 0.94, 'job': 0.93, 'workload': 0.93, 'cleaning': 0.79, 'inspiration': 0.74, 'health': 0.73, 'series': 0.7, 'worries': 0.69, 'kindness': 0.66, 'learning': 0.65, 'writing': 0.56, 'noise': 0.56, 'gratitude': 0.55, 'travel': 0.49, 'tiredness': 0.48, 'stress': 0.41, 'tutorial': 0.4, 'compassion': 0.39, 'reading': 0.38, 'technology': 0.38, 'meeting': 0.34, 'pain': 0.32, 'weather': 0.27, 'deadlines': 0.26, 'social interaction': 0.18, 'friends': 0.17, 'exercise': 0.16, 'happiness': 0.16, 'illness': 0.15, 'pollution': 0.13, 'family': 0.13, 'anxiety': 0.13, 'traffic': 0.1, 'water': 0.1, 'songs': 0.1, 'conflict': 0.09, 'crowds': 0.07, 'streaming': 0.06, 'boredom': 0.05, 'mindfulness': 0.05, 'designing': 0.05, 'negativity': 0.05, 'algorithms': 0.04, 'disrespect': 0.04, 'nature': 0.04, 'entertainment': 0.03, 'podcast': 0.03, 'prayer': 0.03, 'loneliness': 0.03, 'love': 0.03, 'news update': 0.03, 'quran': 0.03, 'documentary': 0.03, 'coding': 0.03, 'walk': 0.02, 'depressio

In [ ]:
classify("my friend met with an accident and died",loop=1)

{'social interaction': 0.96, 'meeting': 0.95, 'friends': 0.92, 'outdoors': 0.89, 'pain': 0.86, 'conflict': 0.81, 'illness': 0.74, 'noise': 0.72, 'worries': 0.64, 'inspiration': 0.54, 'family': 0.45, 'series': 0.44, 'learning': 0.38, 'negativity': 0.37, 'stress': 0.37, 'tutorial': 0.33, 'tiredness': 0.31, 'nature': 0.3, 'compassion': 0.29, 'repair': 0.27, 'news update': 0.26, 'traffic': 0.26, 'writing': 0.25, 'streaming': 0.21, 'depression': 0.2, 'documentary': 0.2, 'anxiety': 0.19, 'water': 0.18, 'hospital': 0.14, 'loneliness': 0.14, 'weather': 0.13, 'workload': 0.13, 'pollution': 0.11, 'disrespect': 0.11, 'travel': 0.1, 'reading': 0.1, 'video posting': 0.1, 'walk': 0.09, 'adventure': 0.08, 'dairy': 0.07, 'love': 0.06, 'prayer': 0.06, 'gratitude': 0.06, 'exercise': 0.06, 'songs': 0.06, 'crowds': 0.05, 'youtube': 0.04, 'video call': 0.04, 'boredom': 0.04, 'reddit': 0.04, 'job': 0.04, 'kindness': 0.04, 'research': 0.04, 'entertainment': 0.04, 'music': 0.03, 'deadlines': 0.03, 'podcast': 